# **Pipeline de ensamble de genomas con nanopore**

Los pasos generales para un ensamble son:

![Pasos](https://raw.githubusercontent.com/sivico26/notebooks/master/Pipeline/Images/steps.png)


El workflow en breve:

- Basecalling
- Basecall QC
- Read-read overlap
- (¿Corrección de error?)
- Ensamble (Guiado o de novo)
- Read mapping
- Polishing (Nanopolish, ¿PoreSeq?)
- Assembly QC

### **Basecalling**

Existen Basecallers basados en HMM(*e.g.* Nanocall) y otros basados en RNN(*e.g.* Scrappie). En comparación las RNN no hacen ninguna suposición de longitud de la secuencia y no son afectados por repeticiones en la secuencia. Aún así, es un reto determinar la longitud real de homopolimeros (*e.g.* AAAAAAAAAAA)

Dentro de los basecallers que se podrían considerar para el pipeline dada una muy buena [comparación](https://github.com/rrwick/Basecalling-comparison) concluimos que __Albacore__ es por ahora la mejor opción disponible. No vale la pena incorporar otros basecallers si no hay diferencias significativas. Ahora, __Guppy__ podría considerarse ya que hace lo mismo que __Albacore__ pero está diseñado para aprovechar GPU, por lo que es mucho más rápido si puede acceder a este recurso (Nota: Guppy no soporta demultiplexado). 

**Nota:** Scrappy y Nanocall (y muy probablemente Albacore), al paralelizar aumentan linealmente su velocidad, para también el uso de memoria

Hay que darle la oportunidad a Chiron? No, por lento

In [ ]:
! read_f

### **Read-read overlap**

La contienda **Minimap vs Graphmap** la gana de lejos Minimap (misma precisión, 2.5x más rápído y usa 4.6x menos memoria), y eso sin contar que minimap ya va en **Minimap2**.
<br>
Por su parte, como lo apunto Heng-Li en su [blog](http://lh3.github.io/2018/04/02/minimap2-and-the-future-of-bwa) BWA-MEM funciona muy bien con los reads cortos de Illumina, y en este campo es cuestionable si Minimap2 lo bate en este sentido, pero para reads largos, Minimap2 es el indicado. 
<br>
En conclusión: ¿Qué algoritmo usar? $\rightarrow$ Distribución de Reads

¿Dónde queda **Minialign**? No está referenciado aún vs Minimap2. Sospechamos que es más rápido y más impreciso, pero hay que evaluarlo.

BWA-MEM: genera SAMs <br>
Minimap2: genera PAFs y SAMs

Minimap es mucho más rápido que BWA-MEM

**Nota:** El multithreading de Minimap2(Curva verde) aumenta la velocidad del algoritmo, pero llega a un máximo y luego la velocidad vuelve a disminuir. Esto ocurre por la sincronización del overhead en los múltiples threads. 

![Multithreading_minimap](https://raw.githubusercontent.com/sivico26/notebooks/master/Pipeline/Images/minimap_parallelizing)

### **Assembly**

**Canu** es ~1096x más lento que **Miniasm**, pero más impreciso también. Esto es por la corrección de error que hace Canu (muy pesada computacionalmente).

Evaluar Canu vs FALCON

### **Polishing**

Es **mandatorio** usar Nanopolish (o en general, hacer polishing), y hacerlo con la opción de ```--methylation-aware dcm,dca```

__Medaka__ es más rápido que __Nanopolish__, aunque corrige menos el error. Debido a que no evalua la raw signal, así que se puede considerar en casos de premura o en los que no se cuente con los fast5.

Nanopolish: solo acepta SAM como entrada y los fast5 **(Revisar actualización)**
<br>
Racon: acepta SAM y PAF, pero requiere los fastq 

Parece que si se va a usar Racon, es mejor usar Minimap para mapear (ligeramente), usar formato SAM mejora usa menos memoria que 

**Nota:** No usar hyperthreading con Nanopolish

## **Notas Generales**

- Si se va a utilizar Nanonpolish, no ensamblar con miniasm (utilizar Canu), termina siendo más lento el proceso completo
- Ensamble rápido $\rightarrow$ Miniasm + Racon
- Ensamble preciso $\rightarrow$ Canu + Nanopolish

## **Criterios para selección de Pipeliner**

- Que siga un estándar
- Que sea fácil de usar
- Que sea "montable" en el biohub
- Que sea fácil de modificar
- Que sea escalable
- Que sea paralizable
- Que utilice checkpoints
- Que esté bien documentado

Basados en esta [lista](https://github.com/pditommaso/awesome-pipeline) y filtrando por herramientas pensadas para bioinformática:

- [Antha](https://www.antha-lang.org/) (Es visual, se ve muy llamativo para edición. Trabaja Java[creo])
- [Biomake](https://github.com/evoldoers/biomake) (Interesante, pero hay que aprender Prolog)
- [BioQueue](https://github.com/liyao001/BioQueue) (Asignación de recursos más eficiente a los programas. No parece un entorno que facilite deasarrollar pipelines)
- [BPipe](https://github.com/ssadedin/bpipe/) (Dicen ser la herramienta más fácil de usar "No Programming knowledge required". No aparenta tener un manejo sencillo de parámetros)
- [Cluster Flow](http://clusterflow.io/) (Similar a BPipe e que se enfocan en facilidad sacrificando poder de desarrollo. Ambos parecen bastante estructurados y manejables una vez construido el pipeline (Pero CF parece más bioinformático, pre-built))
- [Dagr](https://github.com/fulcrumgenomics/dagr)
- [Loom](https://github.com/StanfordBioinformatics/loom)
- [Moa](https://github.com/mfiers/Moa)
- [Nextflow](http://www.nextflow.io/) (Hasta ahora la más atractiva. Entre muchas cosas, portable)
- [Rubra](https://github.com/bjpop/rubra)
- [Ruffus](http://www.ruffus.org.uk/)
- [Snakemake](https://bitbucket.org/johanneskoester/snakemake/wiki/Home)
- [Luigi](https://github.com/spotify/luigi) (Desarrollado por Spotify. Bastante enfocada a Hadoop. Visual.)


| Programa | Modificable | Usable | Estándar | Biohub | Escalable | Checkpoints |
| -- | -- | -- | -- | -- | -- | -- |
| Snakemake | 3 | x | Make | Sí | Sí | Sí |
| Nextflow  | 4 | x | Groovy | Sí | Sí | Sí | 
| Bpipe     | 5 | 5 | Groovy | Sí |  No  | Sí | 
| Luigi     |   |   | Python | Sí | Sí | Sí |
| Cluster Flow  | 5 | 5 | Bash | Sí | Sí | Sí |


| Luigi     |   |   |    |    |    | |
